In [ ]:
# from rapidfuzz.process import extractOne
from difflib import SequenceMatcher
from rapidfuzz.fuzz import ratio
import pandas as pd
import numpy as np
import warnings

# Configuração de exibição.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Ignorando DtypeWarning.
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

In [ ]:
dfs = ['arteris_01_01_2024_v1','bndes_01_01_2024_v1', 'braskem_01_01_2024_v1', 'cury_01_01_2024_v1',
       'grupo_vamos_01_01_2024_v1', 'grupo_gps_01_01_2024_v1', 'IRB_Brasil_01_01_2024_v1', 'JSL_01_01_2024_v1',
       'Rodobens_01_01_2024_v1', 'smartfit_01_01_2024_v1', 'taurus_01_01_2024_v1', 'wilson_01_01_2024_v1']

for current_df in dfs:
    sheet_name = current_df

    # Read the Excel sheets for metrics and golden collection data
    df_metrics = pd.read_excel('../Data/calculo_metricas.xlsx', sheet_name=sheet_name, dtype={'valor': str, 'page': str, 'titulo_coluna': str}, engine='openpyxl')
    df_gold_collection = pd.read_excel('../Data/COLECAO_DOURADA.xlsx', sheet_name=sheet_name, dtype={'valor': str, 'page': str, 'titulo_coluna': str}, engine='openpyxl')

    def compare(df_metrics, df_gold_collection):
        max_length = max(len(df_metrics), len(df_gold_collection))
        word_indices = {
            0: 'id_quadro',
            1: 'titulo_quadro',
            2: 'page',
            3: 'warning_quadro',
            4: 'titulo_coluna',
            5: 'warning_col',
            6: 'titulo_linha',
            7: 'warning_linha',
            8: 'valor',
            9: 'warning_value'
        }

        filtered_dfs = []

        for num in range(max_length):
            quadro_id = f'quadro_{num}'

            # Filter rows based on the same id_quadro
            df_metrics_temp = df_metrics[df_metrics['id_quadro'] == quadro_id].reset_index(drop=True)
            df_gold_collection_temp = df_gold_collection[df_gold_collection['id_quadro'] == quadro_id].reset_index(drop=True)

            # Add 'status' and 'error_reason' columns
            df_gold_collection_temp['status'] = ''
            df_gold_collection_temp['error_reason'] = '-'

            for i in range(len(df_gold_collection_temp)):
                arr1 = df_gold_collection_temp.iloc[i].astype(str)
                best_similarity = 0
                found = False

                for j in range(len(df_metrics_temp)):
                    arr2 = df_metrics_temp.iloc[j].astype(str)

                    # Index requiring exact comparison
                    numeric_index = 8

                    # Similarity comparison for each pair of rows
                    string_indices = [0, 1, 3, 4, 5, 6, 7, 8, 9]

                    # Filter elements from arr1 and arr2 considering only the columns specified
                    arr1_filtered = [str(arr1[i]) for i in string_indices]
                    arr2_filtered = [str(arr2[i]) for i in string_indices]

                    # Calculate similarity between the two arrays
                    similarity1 = SequenceMatcher(None, arr1, arr2).ratio()

                    # Find the most similar array to compare differences
                    if similarity1 > best_similarity:
                        best_similarity = similarity1
                        closest_line = arr2

                    if arr1[numeric_index] == arr2[numeric_index]:
                        # Similarity comparison for the other indices
                        similarity2 = [round(ratio(arr1_filtered[i], arr2_filtered[i]), 2) for i in range(min(len(arr1_filtered), len(arr2_filtered)))]

                        # Check if all similarities are greater than 94, except for the numeric index that must be exactly the same
                        if all(similarity2 >= 94 for i, similarity2 in enumerate(similarity2) if i != string_indices.index(numeric_index)):
                            df_metrics_temp.at[i, 'status'] = 'Not Error'
                            df_gold_collection_temp.at[i, 'status'] = 'Not Error'
                            df_gold_collection_temp.at[i, 'error_reason'] = ''
                            found = True
                            break
                if not found:
                    df_gold_collection_temp.at[i, 'status'] = 'Error'

                    # Compare element by element for the specified indices
                    different_elements = []
                    for index in string_indices:
                        elem1 = arr1[index]
                        elem2 = closest_line[index]
                        if elem1 != elem2 and index != 1:
                            word_index = word_indices[index]
                            different_elements.append(word_index)

                    error_reason = ", ".join(different_elements)
                    df_gold_collection_temp.at[i, 'error_reason'] = error_reason

            filtered_dfs.append(df_gold_collection_temp)

        return pd.concat(filtered_dfs, ignore_index=True)

    def save_to_csv(df_gold_collection, sheet_name):
        # Save the DataFrame to a CSV file
        df_gold_collection.to_csv(f'../reviews/{sheet_name}.csv', index=False)

    # Call the functions
    df_gold_collection_filtered = compare(df_metrics, df_gold_collection)
    save_to_csv(df_gold_collection_filtered, sheet_name)
